In [ ]:
import numpy as np
from astropy.io import fits
import os
import re
from xattr import xattr

from src.hdu.cubes.cube import Cube
from src.hdu.arrays.array_3d import Array3D
from src.headers.header import Header
from src.coordinates.equatorial_coords import RA, DEC

The following document helped to understang the Global Sinusoidal (GLS) projection : [Multi-Beam FITS Raw Data Format, page 15](https://fits.gsfc.nasa.gov/registry/mbfits/APEX-MPI-ICD-0002-R1_66.pdf)

In [50]:
def translate_id(id: str) -> np.ndarray:
    """ 
    Translates a string id into a vector.
    """
    translation = {
        "N" : np.array([ 0, 1]),
        "S" : np.array([ 0,-1]),
        "E" : np.array([-1, 0]),
        "W" : np.array([ 1, 0])
    }

    # Define the pattern to match letter followed by digits
    pattern = re.compile(r"([A-Za-z])(\d*)")
    matches = pattern.findall(id)
    result = np.array([0, 0])
    for letter, number in matches:
        if letter.upper() not in list(translation.keys()):
            continue
        if number == "":
            number = 1
        else:
            number = int(number)
        result += translation[letter.upper()] * number
    
    return result

def set_region(cube_data: np.ndarray, center: np.ndarray, region: fits.HDUList):
    """
    Sets a region of a given cube.
    """
    smart_slice = lambda start, length: slice(int(start), int(start+length))
    cube_data[
        :int(region.header["NAXIS3"]),
        smart_slice(center[1]-region.header["CRPIX2"], region.header["NAXIS2"]),
        smart_slice(center[0]-region.header["CRPIX1"], region.header["NAXIS1"])
    ] = region.data

def get_cleaned_header(header: Header) -> Header:
    """
    Filters the given header to remove any invalid keywords.
    """
    valid_cards = []
    valid_keywords = [
        "SIMPLE", "BITPIX", "NAXIS", "NAXIS1", "NAXIS2", "NAXIS3", "BUNIT", "EQUINOX", "CRPIX1", "CRPIX2", "CROTA1",
        "CROTA2", "CRVAL1", "CRVAL2", "CTYPE1", "CTYPE2", "CDELT1", "CDELT2", "CRPIX3", "CROTA3", "CRVAL3", "CTYPE3",
        "CDELT3", "BUNIT", "OBSERVER", "LINE", "EQUINOX", "VELO-LSR"
    ]
    for card in header.cards:
        if card.keyword in valid_keywords:
            valid_cards.append(fits.Card(
                keyword=card.keyword,
                value=card.value,
                comment=card.comment
            ))
    
    return Header(valid_cards)

def set_header(target_cube: Cube, reference_cube: Cube):
    target_cube.header["CTYPE1"] = reference_cube.header["CTYPE1"]
    target_cube.header["CTYPE2"] = reference_cube.header["CTYPE2"]
    target_cube.header["CDELT1"] = reference_cube.header["CDELT1"]
    target_cube.header["CDELT2"] = reference_cube.header["CDELT2"]
    target_cube.header["CRPIX1"] = reference_cube.header["CRPIX1"]
    target_cube.header["CRPIX2"] = reference_cube.header["CRPIX2"]
    target_cube.header["CRVAL1"] = reference_cube.header["CRVAL1"]
    target_cube.header["CRVAL2"] = reference_cube.header["CRVAL2"]
    return target_cube

def build_cube(prefix: str, ref_cube: Cube) -> Cube:
    # Create an empty Cube of arbitrary size
    cube_data = np.full((10000, 200, 200), np.NAN)
    files = os.listdir(f"data/Loop4_co/13co_spectrums/{prefix}")

    # Create an arbitrary center
    center = np.array([100, 100])

    # Iterate over the files in the directory to get only the cubes
    # Individual spectrums are added afterwards as the addition of a spectrum, then a cube at the same position
    # overwrites the previous spectrum
    first_file = True
    for file in filter(lambda f: not f.endswith("-s.fits"), files):
        if file == "N4S4.fits": continue        # Do not include this file as it has an inconsistent spectral resolution
        subregion = fits.open(f"data/Loop4_co/13co_spectrums/{prefix}/{file}")[0]
        subregion_id = file[len(prefix):-5]
        if first_file:
            cube_header = fits.open(f"data/Loop4_co/13co_spectrums/{prefix}/{file}")[0].header
            CRPIX_offset = - 2 + translate_id(subregion_id)*3
            cube_header["CRPIX1"] += center[0] + CRPIX_offset[0]
            cube_header["CRPIX2"] += center[1] + CRPIX_offset[1]
            first_file = False
        set_region(cube_data, center + translate_id(subregion_id)*3, subregion)

    # Loop on all the spectrum files, if any
    for file in filter(lambda f: f.endswith("-s.fits"), files):
        # Filter out grey tags (only relevant for Loop4p spectrums)
        try:
            if (xattr(f"data/Loop4_co/13co_spectrums/{prefix}/{file}")['com.apple.FinderInfo'][9] >> 1 & 7) == 1:
                # Files with the grey tag on top will be skipped
                continue
        except KeyError:
            # The file does not have any tag -> will be considered by default
            pass

        subregion = fits.open(f"data/Loop4_co/13co_spectrums/{prefix}/{file}")[0]
        subregion_id = file[len(prefix):-5]
        subregion_center = center + translate_id(subregion_id[:-2])*3
        cube_data[
            :int(subregion.header["NAXIS1"]),
            subregion_center[1]-1,
            subregion_center[0]-1
        ] = subregion.data

    cube_data[cube_data == 0] = np.NAN
    cube = Cube(Array3D(cube_data), get_cleaned_header(cube_header))
    cube.header["OBJECT"] = f"Loop4{prefix}"

    # Crop the cube to the same dimensions
    x_lower_limit = round(cube.header["CRPIX1"] - ref_cube.header.get_coordinate(cube.header["CRVAL1"], 2))
    y_lower_limit = round(cube.header["CRPIX2"] - ref_cube.header.get_coordinate(cube.header["CRVAL2"], 1))
    x_upper_limit = round(x_lower_limit + ref_cube.header["NAXIS1"])
    y_upper_limit = round(y_lower_limit + ref_cube.header["NAXIS2"])
    z_upper_limit = round(np.where(np.all(np.isnan(cube.data), axis=(1,2)))[0][0])

    cube = cube[:z_upper_limit, y_lower_limit:y_upper_limit, x_lower_limit:x_upper_limit]

    # The last step is to perfectly align the header using the 12CO aligned cube (also change the projection CAR)
    set_header(cube, ref_cube)
    cube.save(f"data/Loop4_co/{prefix}/13co/Loop4{prefix}_13co.fits")

In [ ]:
build_cube("N1", ref_cube=Cube.load("data/Loop4_co/N1/12co/Loop4N1_wcs.fits"))

In [ ]:
build_cube("N2", ref_cube=Cube.load("data/Loop4_co/N2/12co/Loop4N2_wcs.fits"))

In [ ]:
build_cube("N4", ref_cube=Cube.load("data/Loop4_co/N4/12co/Loop4N4_wcs.fits"))

In [ ]:
# N4S4
N4S4 = Cube.load("data/Loop4_co/13co_spectrums/N4/N4S4.fits")
N4S4.header = get_cleaned_header(N4S4.header)
N4S4.save("data/Loop4_co/N4/13co/N4S4.fits")

In [51]:
build_cube("p", ref_cube=Cube.load("data/Loop4_co/p/12co/Loop4p_wcs.fits"))

In [ ]:
print(RA(1.2683333439634e+02))
print(DEC(6.0125000961653e+01))